In [ ]:
import pandas as pd
import seaborn as sns
sns.set()

import datetime
import os

from sklearn.preprocessing import LabelEncoder, MinMaxScaler

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical

logs_base_dir = "./logs"
logdir = os.path.join(logs_base_dir, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

chk_base_dir = "./checkpoints"

In [ ]:
test_data = pd.read_csv('test_data.csv')

In [ ]:
def handle_missing_data(_df):
    df=_df
    df.dropna(inplace=True)
    return df

def feature_eng(_df):
    df=_df
    df['avg_CC_1hop'] = df['sum_1hop_current_charge']/df['size_1hop']
    df['avg_CC_2hop'] = df['sum_2hop_current_charge']/df['size_2hop']
    df['avg_CC_3hop'] = df['sum_3hop_current_charge']/df['size_3hop']
    df['avg_EF_1hop'] = df['sum_1hop_energy_fraction']/df['size_1hop']
    df['avg_EF_2hop'] = df['sum_2hop_energy_fraction']/df['size_2hop']
    df['avg_EF_3hop'] = df['sum_3hop_energy_fraction']/df['size_3hop']
    df = df.drop([
        'sum_1hop_current_charge', 
        'sum_2hop_current_charge', 
        'sum_3hop_current_charge',
        'sum_1hop_energy_fraction',
        'sum_2hop_energy_fraction',
        'sum_3hop_energy_fraction',
        'sum_1hop_flow_counter',
        'sum_2hop_flow_counter',
        'sum_3hop_flow_counter',
        'sum_1hop_initial_charge',
        'sum_2hop_initial_charge',
        'sum_3hop_initial_charge'
    ], axis='columns')
    return df

In [ ]:
X_test = test_data
X_test = feature_eng(X_test)

In [ ]:
y_test = to_categorical(X_test['label_energy_fraction'].values)

X_test = X_test.drop(['label_current_charge', 'label_energy_fraction', 'label_flow_count', 'label_initial_charge'], axis='columns')

le = LabelEncoder()
X_test['frame_type'] = le.fit_transform(X_test['frame_type'])

In [ ]:
# Returns a short sequential model
def create_model(sample_input): # numpy array so we can use .shape
    model = tf.keras.Sequential([
        layers.Dense(64, activation='softmax', input_shape=sample_input.shape),
        layers.Dense(64, activation='softmax'),
        layers.Dense(1, activation='softmax')
    ])

    # from my initial tests
    model.compile(optimizer=tf.train.RMSPropOptimizer(0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    # from another tf.keras tutorial
#     model.compile(optimizer=tf.keras.optimizers.Adam(),
#                 loss=tf.keras.losses.sparse_categorical_crossentropy,
#                 metrics=['accuracy'])

    return model

In [ ]:
print(X_test.values[0].shape)
model = create_model(X_test.values[0])

In [ ]:
right = 0
wrong = 0

for index, row in X_test.iterrows():
    print(row.values.reshape((1, 11)))
    prediction = model.predict(row.values.reshape((1, 11)) )
    l = y_test[index][0]
    print(prediction, l)
    if prediction == y_test[index][0]:
        right += 1
    else:
        wrong +=1
print("--- 3 hops model ---")
print("Before loading weights")
print("Correct prediction = ", right)
print("Wrong prediction = ", wrong)

In [ ]:
model.load_weights(os.path.join(chk_base_dir,'3hops'))

In [ ]:
right = 0
wrong = 0
for index, row in X_test.iterrows():
    print(row.values.shape)
    prediction = model.predict(row.values)
    if prediction == y_test.values[index]:
        right += 1
    else:
        wrong +=1
print("After loading weights")
print("Correct prediction = ", right)
print("Wrong prediction = ", wrong)